# Convert Phase 2 well data
Locher Environmental recorded data from multiple transducers in two adjacent wells near SLC 39A at Kennedy Space Center (KSC), between March and November, 2022. This was part of a seismo-acoustic erosion pilot experiment. During phase 2 of the experiment, which began on July 21st, 2022, vibrating wire sensors were used and found to give more accurate water levels. These data were captured on Campbell Scientific dataloggers and recorded in TOB3 binary file format. The purpose of this notebook is to:
- read these files
- apply calibration equations
- write to pickle files (Python binary files that serialize the data variables)
Calibration constants are defined in the transducers dataframe (section 2). These are copied from the file "A Pz Linear Gage Calc_NASA Sensors.xls". Other metadata included in this dataframe come from sections 6 and 7 in the file "2022-12-03_ Field Sheet for Deployment of Groundwater Equipment at NASA_Part_II.pdf".

Note on *TIME ZONES*: 
- Local time is used in Campbell Scientific binary files, and files converted to Pickle.
- UTC is used in MiniSEED files, to match seismo-acoustic data.

Note on *UNITS*:
- Pi

To do:
- create a lookup table, matching file name, start time, end time, and SEED trace-ID.


# 1. Imports, path variables, and function definitions

In [ ]:
# raw data on April 1, 2022 from 16:10 to 16:40 UTC. Launch from SLC40 at 16:24 UTC, watched from Titusville
%run header.ipynb


# 2. Read raw data files, convert to CSV files using CS Python libraries

In [ ]:
# Read Campbell Scientific TOB3 file
write_csv = False
write_raw = False
keep_existing = False
allcolumns = []
lod = []

# Generate complete list of TOB3 files (raw TOB3 files from CS dataloggers)
tob3files = LLE.list_loggernet_tob3files(paths['TOB3_DIR'])

for filenum, tob3file in enumerate(tob3files):
    tob3base = os.path.basename(tob3file)
    print('File %d of %d: %s' % ((filenum+1), len(tob3files), tob3file))
    dirname = os.path.basename(os.path.dirname(tob3file))
    csvfile = tob3file.replace('.dat','.py_csv')
    if os.path.isfile(csvfile) & keep_existing:
        print('- Already DONE')
    else:
        print('- Reading')
        data, meta = campbell.read_cs_files(tob3file, forcedatetime=False,
                      bycol=True, quiet=True, metaonly=False)
        print('- converting to dataframe')
        try: # adding because one time meta was just a bool and not subscriptable
            df = pd.DataFrame(columns=meta[2]) 
        except:
            continue
        for c in range(len(meta[2])):
            df[meta[2][c]] = data[c]
            if not meta[2][c] in allcolumns:
                allcolumns.append(meta[2][c])       
        print('- writing raw data to %s' % csvfile)
        df.to_csv(csvfile)



# 3. Read raw CSV files, apply calibration equations, write out to corrected CSV files

In [ ]:
# Generate complete list of .raw_csv files (raw TOB3 files from CS dataloggers converted by Python library above)
pycsvfiles = LLE.list_loggernet_csvfiles(paths['TOB3_DIR'], ext='.py_csv')
        
LLE.correct_csvfiles(pycsvfiles, paths, converted_by_LoggerNet=False, MAXFILES=None, keep_existing=True)